In [1]:
import networkx as nx
import time
import math

In [2]:
def read_graph(path):
    """ Read 'edges.txt.gz' into a networkx **undirected** graph.
    Done for you.
    Returns:
      A networkx undirected graph.
    """
    return nx.read_edgelist(path)
# graph=read_graph('data/user_friends.txt')

In [3]:
def girvan_newman(G, depth=0):
    """ Recursive implementation of the girvan_newman algorithm.
    See http://www-rohan.sdsu.edu/~gawron/python_for_ss/course_core/book_draft/Social_Networks/Networkx.html
    
    Args:
    G.....a networkx graph

    Returns:
    A list of all discovered communities."""
    #find the best edge
    def find_best_edge(g):
        edge_betweeness = nx.edge_betweenness_centrality(g)
        
        return sorted(edge_betweeness.items(), key=lambda x: x[1], reverse=True)[0][0]
    
    if G.order() == 1:
        return [G.nodes()]
    
    # Each component is a separate community. We cluster each of these.
    components = [compo for compo in nx.connected_component_subgraphs(G)]

    while len(components) == 1:
        #find best edge to remove
        edge_to_remove = find_best_edge(G)
        #remove the best edge
        G.remove_edge(*edge_to_remove)
        
        components = [compo for compo in nx.connected_component_subgraphs(G)]

    result = [compo.nodes() for compo in components]

    for c in components:
        result.extend(girvan_newman(c, depth + 1))

    return result
# communities=girvan_newman(graph, depth=0)

In [4]:
def clusterAnswers(result):
    opFile=open("data/clusterAnswers.txt","w+")
    resuldDict={}
    
    print("total Communities: %d"%len(result))
    resuldDict['totalCommunities']=len(result)

    totalCandidate=0
    candidateFile=open("data/candidates.txt","r")
    for c in candidateFile:
        totalCandidate+=1
    
    
    totalFriends=0
    friendsFile=open("data/user_friends.txt","r")
    for f in friendsFile:
        totalFriends+=1
    
    totalUsers=totalCandidate+totalFriends
    resuldDict['totalUsers']=totalUsers
    print("Total Users :%d"%totalUsers)
    finalList=sorted(result, key=lambda x:len(x))[::-1][:50]
    memebers=[]
    totalCommunities=len(finalList)
    for i in finalList:
        memebers.extend(i)
    resuldDict['top50Communities']=len(result)
    print("Top 50 communities: %d"%totalCommunities)
    resuldDict['top50Communities']=totalCommunities
    print("Average users per communities: %d"%math.ceil(len(memebers)/totalCommunities))
    resuldDict['avgUsersPerComunnities']=math.ceil(len(memebers)/totalCommunities)
    flag=True
    for data in resuldDict:
        if flag:
            opFile.write(data+":"+str(resuldDict[data]))
            flag=False
        else:
            opFile.write("\n"+data+":"+str(resuldDict[data]))
    opFile.close()

In [5]:
def main():
    """
    FYI: This takes ~10-15 seconds to run on my laptop.
    """
    startTime = time.clock()
    
    #Read User and friends from the user_friends.txt
    #Create graph for each user with respective friends
    graph=read_graph('data/user_friends.txt')
    
    #Community detection using Girvan_newman algorithm
    communities=girvan_newman(graph, depth=0)
    
    #write clustering answers to the file
    clusterAnswers(communities)
    
    endTime =  time.clock()
    
    print("Total running time :%f"%(endTime - startTime))
    
if __name__ == '__main__':
    main()

total Communities: 918
Total Users :401
Top 50 communities: 50
Average users per communities: 26
Total running time :0.527867
